In [1]:
import os
import geopandas as gpd
import pandas as pd
from shapely import wkt

In [2]:
data_path = r'D:\Projects\superparcels\data\abreunig_temp_sp_sample_08013.csv'

In [3]:
data_dir = r'D:\Projects\superparcels\data'

In [4]:
df = pd.read_csv(data_path)
df.head()

,state_code,cnty_code,OWNER,std_addr,geometry
0,8,13,JAMES HHICKMAN,1028 SKI RD,"POLYGON((-105.529203196417 40.1819162319136, -..."
1,8,13,GERARDGLOISTEN,57 LONGMONT DAM RD,"POLYGON((-105.311374265349 40.2441798158087, -..."
2,8,13,JOYSE P CYUEN,1003 STEAMBOAT VALLEY RD,"POLYGON((-105.275244744667 40.2458552933456, -..."
3,8,13,WILLIAM JSIVY,5595 UTE HWY,"POLYGON((-105.22418535217 40.2142450231427, -1..."
4,8,13,CHRISTOPHER JCARLSTON,11510 EAGLE SPRINGS TRL,"POLYGON((-105.239738403216 40.1836815336225, -..."


In [5]:
gdf = gpd.GeoDataFrame(df, geometry=df['geometry'].apply(wkt.loads), crs=4326)
gdf.to_file(os.path.join(data_dir, 'sp_sample_08013.shp'))

Clean parcel urban and remove duplicate geometries for clustering POC

In [6]:
input_dir = r'D:\Projects\superparcels\data\urban'

In [7]:
df = gpd.read_file(os.path.join(input_dir, 'sp_sample_08013_urban.shp'))

In [8]:
df.head(3)

,state_code,cnty_code,OWNER,std_addr,geometry
0,8,13,ALEAKOOTZ,100 RIDGE DR,"POLYGON ((-105.30127 40.06144, -105.30126 40.0..."
1,8,13,MARIJETERELLEN,501 EVERGREEN AVE,"POLYGON ((-105.28974 40.03127, -105.28974 40.0..."
2,8,13,DENNIS VMARTIN,2940 18TH ST,"POLYGON ((-105.27360 40.02916, -105.27359 40.0..."


In [9]:
#df.dissolve(by='OWNER').to_file(os.path.join(input_dir, 'sp_sample_08013_urban_dissolved.shp'))

In [10]:
#df = df.dissolve(by='OWNER').explode(ignore_index=False).reset_index()

In [23]:
# Identify duplicate owners, addresses, and geometries
df['duplicate_owner'] = df.duplicated(subset=['OWNER'], keep=False)
df['duplicate_address'] = df.duplicated(subset=['std_addr'], keep=False)
df['duplicate_geometry'] = df.duplicated(subset=['geometry'], keep=False)

# Create a classification column based on duplication status (with geometry)
df['classification'] = df.apply(
    lambda row: (
        'Class1: Duplicate Owner, Address & Geometry' if row['duplicate_owner'] and row['duplicate_address'] and row['duplicate_geometry'] else
        'Class2: Duplicate Owner & Address, Unique Geometry' if row['duplicate_owner'] and row['duplicate_address'] and not row['duplicate_geometry'] else
        'Class3: Duplicate Owner & Unique Address, Geometry' if row['duplicate_owner'] and not row['duplicate_address'] and not row['duplicate_geometry'] else
        'Class4: Unique Owner & Duplicate Address & Geometry' if not row['duplicate_owner'] and row['duplicate_address'] and row['duplicate_geometry'] else
        'Class5: Unique Owner & Address, Duplicate Geometry' if not row['duplicate_owner'] and not row['duplicate_address'] and row['duplicate_geometry'] else
        'Class6: Unique Owner & Address & Geometry'
    ), axis=1
)

# Count the occurrences of each classification
matrix = df['classification'].value_counts()

print(matrix)


classification
Class6: Unique Owner & Address & Geometry              14822
Class5: Unique Owner & Address, Duplicate Geometry      5787
Class1: Duplicate Owner, Address & Geometry             3211
Class3: Duplicate Owner & Unique Address, Geometry      2111
Class2: Duplicate Owner & Address, Unique Geometry       434
Class4: Unique Owner & Duplicate Address & Geometry       16
Name: count, dtype: int64


In [24]:
# create clasification codes
df['classification_code'] = df['classification'].apply(lambda x: x.split(':')[0][5])
df['classification_code'] = df['classification_code'].astype(int)
df['classification_code'].value_counts()

classification_code
6    14822
5     5787
1     3211
3     2111
2      434
4       16
Name: count, dtype: int64

In [ ]:
df.head(3)

In [ ]:
class_code = 2
code = df[df['classification_code'] == class_code]
code.to_file(os.path.join(input_dir, f'sp_sample_08013_code{class_code}.shp'))

In [ ]:
cluster_canidate_codes = [2,3] # classes after dissolving
cluster = df[df['classification_code'].isin(cluster_canidate_codes)]
cluster.to_file(os.path.join(input_dir, f'sp_sample_08013_cluster_canidates.shp'))